In [15]:
! pip install transformers datasets py7zr evaluate rouge_score accelerate>=0.20.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.1/258.1 kB 5.2 MB/s eta 0:00:00


In [1]:
from random import randrange

from datasets import load_dataset, concatenate_datasets
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments
from transformers import pipeline
import evaluate
import nltk
import numpy as np
from nltk.tokenize import sent_tokenize
nltk.download("punkt")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

# Dataset

We are going to fine-tune the [Flan-T5](https://huggingface.co/google/flan-t5-small) model to summarize dialogue/ chat threads similar to what is done in the ChatGPT interface. We will use the [SAMSum](https://huggingface.co/datasets/samsum) dataset. The SAMSum dataset contains about 16k messenger-like conversations with summaries. Conversations were created and written down by linguists fluent in English.

(Thanks to [Philipp Schmid](https://www.philschmid.de/) for great examples of this task that were adapted for this notebook)

In [2]:
dataset = load_dataset("samsum")

print(f"Train dataset size: {len(dataset['train'])}")
print(f"Test dataset size: {len(dataset['test'])}")

Train dataset size: 14732
Test dataset size: 819


In [3]:
dataset['train'][0]

{'id': '13818513',
 'dialogue': "Amanda: I baked  cookies. Do you want some?\r\nJerry: Sure!\r\nAmanda: I'll bring you tomorrow :-)",
 'summary': 'Amanda baked cookies and will bring Jerry some tomorrow.'}

In [4]:
#pick a random sample to evaluate the model later
sample = dataset['test'][randrange(len(dataset["test"]))]
sample

{'id': '13731240',
 'dialogue': "Ken: Hey, how are you doing today?\r\nEllen: Fine. How are you?\r\nKen: Not doing well at all today. Really depressed. Frustrated. Stressed out.\r\nEllen: Oh no!\r\nKen: Really bad day yesterday carrying over to today.\r\nEllen: Can I do anything?\r\nKen: I'll be fine. Just overwhelmed and burned out.\r\nEllen: Sorry!\r\nKen: Sorry, don't mean to bring you down.\r\nEllen: You didn't, just worried about you!",
 'summary': 'Ken is having some bad days.'}

# Pre-process data

In [5]:
# We will use the Flan T5 tokenizer to help us pre-process the data.
model_id="google/flan-t5-small"

# Load tokenizer of Flan-T5
tokenizer = AutoTokenizer.from_pretrained(model_id)

In [6]:
# The maximum total input sequence length after tokenization.
# Sequences longer than this will be truncated, sequences shorter will be padded.
tokenized_inputs = concatenate_datasets([
    dataset["train"],
    dataset["test"]]).map(lambda x: tokenizer(x["dialogue"], truncation=True),
    batched=True,
    remove_columns=["dialogue", "summary"])
max_source_length = max([len(x) for x in tokenized_inputs["input_ids"]])
print(f"Max source length: {max_source_length}")

# The maximum total sequence length for target text after tokenization.
# Sequences longer than this will be truncated, sequences shorter will be padded."
tokenized_targets = concatenate_datasets([
    dataset["train"],
    dataset["test"]]).map(lambda x: tokenizer(x["summary"], truncation=True),
    batched=True,
    remove_columns=["dialogue", "summary"])
max_target_length = max([len(x) for x in tokenized_targets["input_ids"]])
print(f"Max target length: {max_target_length}")

Max source length: 512


Map:   0%|          | 0/15551 [00:00<?, ? examples/s]

Max target length: 95


In [7]:
def preprocess_function(sample,padding="max_length"):

    # Add prefix to the input prompt for t5
    inputs = ["summarize: " + item for item in sample["dialogue"]]

    # tokenize inputs
    model_inputs = tokenizer(inputs, max_length=max_source_length,
                             padding=padding, truncation=True)

    # Tokenize targets with the `text_target` keyword argument
    labels = tokenizer(text_target=sample["summary"],
                       max_length=max_target_length, padding=padding,
                       truncation=True)

    # If we are padding here, replace all tokenizer.pad_token_id in the labels
    # by -100 when we want to ignore padding in the loss.
    if padding == "max_length":
        labels["input_ids"] = [
            [(l if l != tokenizer.pad_token_id else -100) for l in label] for label in labels["input_ids"]
        ]

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [8]:
tokenized_dataset = dataset.map(preprocess_function, batched=True,
                                remove_columns=["dialogue", "summary", "id"])
print(f"Keys of tokenized dataset: {list(tokenized_dataset['train'].features)}")

Map:   0%|          | 0/819 [00:00<?, ? examples/s]

Keys of tokenized dataset: ['input_ids', 'attention_mask', 'labels']


# Base/ Foundation Model

In [9]:
# load model from the hub
model = AutoModelForSeq2SeqLM.from_pretrained(model_id)

Before we fine-tune, let's see how the model performs on our sample!

In [10]:
base_summarizer = pipeline("summarization", model=model, tokenizer=tokenizer, device=0)

# print dialogue and reference summary
print(f"dialogue: \n{sample['dialogue']}\n---------------")
print(f"reference summary: \n{sample['summary']}\n---------------")

# print model summary
base_result = base_summarizer(sample["dialogue"])
print(f"base flan-t5-base summary:\n{base_result[0]['summary_text']}")

Your max_length is set to 200, but your input_length is only 109. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=54)


dialogue: 
Ken: Hey, how are you doing today?
Ellen: Fine. How are you?
Ken: Not doing well at all today. Really depressed. Frustrated. Stressed out.
Ellen: Oh no!
Ken: Really bad day yesterday carrying over to today.
Ellen: Can I do anything?
Ken: I'll be fine. Just overwhelmed and burned out.
Ellen: Sorry!
Ken: Sorry, don't mean to bring you down.
Ellen: You didn't, just worried about you!
---------------
reference summary: 
Ken is having some bad days.
---------------
base flan-t5-base summary:
Ken is not doing well today. He is depressed and stressed out. Ellen will be fine. Ken will take care of her. Ellen is worried about her.


# Fine-tune

In [11]:
# We will use the Rouge metric for evaluations. ROUGE, or Recall-Oriented
# Understudy for Gisting Evaluation, is a set of metrics and a software package
# used for evaluating automatic summarization and machine translation software
# in natural language processing. The metrics compare an automatically produced
# summary or translation against a reference or a set of references
# (human-produced) summary or translation.
metric = evaluate.load("rouge")

# helper function to postprocess text
def postprocess_text(preds, labels):

    preds = [pred.strip() for pred in preds]
    labels = [label.strip() for label in labels]

    # rougeLSum expects newline after each sentence
    preds = ["\n".join(sent_tokenize(pred)) for pred in preds]
    labels = ["\n".join(sent_tokenize(label)) for label in labels]

    return preds, labels

def compute_metrics(eval_preds):

    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Some simple post-processing
    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    result = {k: round(v * 100, 4) for k, v in result.items()}
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    result["gen_len"] = np.mean(prediction_lens)
    return result

In [12]:
# we want to ignore tokenizer pad token in the loss
label_pad_token_id = -100

# Data collator
data_collator = DataCollatorForSeq2Seq(
    tokenizer,
    model=model,
    label_pad_token_id=label_pad_token_id,
    pad_to_multiple_of=8
)

In [13]:
# Define training args
training_args = Seq2SeqTrainingArguments(
    output_dir="flan-t5-samsum",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    predict_with_generate=True,
    fp16=False, # Overflows with fp16
    learning_rate=5e-5,
    num_train_epochs=5,
    # logging & evaluation strategies
    logging_dir=f"flan-t5-samsum/logs",
    logging_strategy="steps",
    logging_steps=500,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=2,
    load_best_model_at_end=True,
)

# Create Trainer instance
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    compute_metrics=compute_metrics,
)

In [14]:
# Start training
trainer.train()

You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
1,1.817500,1.663650,43.935900,20.084400,34.535800,40.576600,36.269841
2,1.735000,1.642660,44.049800,20.247700,34.637500,40.495400,36.081807
3,1.696300,1.634848,44.828500,21.007400,35.559900,41.364500,36.521368
4,1.664400,1.633715,44.564500,20.676000,35.222300,41.063300,36.463980
5,1.637100,1.632018,44.802800,20.714300,35.410700,41.338800,36.553114


TrainOutput(global_step=9210, training_loss=1.7139073905157862, metrics={'train_runtime': 1807.9254, 'train_samples_per_second': 40.743, 'train_steps_per_second': 5.094, 'total_flos': 1.369269457649664e+16, 'train_loss': 1.7139073905157862, 'epoch': 5.0})

 # Evaluate

In [15]:
# Evaluate using the same trainer.
trainer.evaluate()

{'eval_loss': 1.6320182085037231,
 'eval_rouge1': 44.8028,
 'eval_rouge2': 20.7143,
 'eval_rougeL': 35.4107,
 'eval_rougeLsum': 41.3388,
 'eval_gen_len': 36.553113553113555,
 'eval_runtime': 113.1792,
 'eval_samples_per_second': 7.236,
 'eval_steps_per_second': 0.91,
 'epoch': 5.0}

# Try it out!

In [25]:
model_base = AutoModelForSeq2SeqLM.from_pretrained(model_id)

In [30]:
# load model and tokenizer from with pipeline
finetuned_summarizer = pipeline("summarization", model=model, tokenizer=tokenizer, device=0)
base_summarizer = pipeline("summarization", model=model_base, tokenizer=tokenizer, device=0)

# select a random test sample
sample = dataset['test'][randrange(len(dataset["test"]))]
print(f"dialogue: \n{sample['dialogue']}\n---------------")
print(f"reference summary: \n{sample['summary']}\n---------------")

# summarize dialogue
base_result = base_summarizer(sample["dialogue"])
finetuned_result = finetuned_summarizer(sample["dialogue"])

print(f"base model summary: \n{base_result[0]['summary_text']}\n---------------")
print(f"finetuned model summary:\n{finetuned_result[0]['summary_text']}")

Your max_length is set to 200, but your input_length is only 95. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=47)


dialogue: 
Tomas: Has anybody received the grant yet?
Sierra: no, not yet
Jeremy: I haven't checked even
Tomas: I'm completely broke
Tomas: checking my bank account every hour
Tomas: but nothing's happening there
Sierra: lol
Sierra: be patient. If you need money I can lend you some, don't worry
Tomas: Thanks, I hope they'll arrive any minute
---------------
reference summary: 
Tomas, Sierra and Jeremy have still not received the grant. Tomas is broke and is checking his bank account every hour. Sierra offers to lend him some money.
---------------


Your max_length is set to 200, but your input_length is only 95. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=47)


base model summary: 
Tomas has received the grant. Jeremy hasn't checked yet. He's checking his bank account every hour. He will lend him some money if he needs it.
---------------
finetuned model summary:
Sierra hasn't received the grant yet. Tomas is broke. Sierra will lend him some money if he needs it. 
